文本分类是自然语言处理的一个常见任务，它把一段不定长的文本序列变换为文本的类别。同搜索近义词和类比词一样，文本分类也属于词嵌入的下游应用.

### RNN
每个词先通过嵌入层得到特征向量。然后，我们使用双向循环神经网络对特征序列进一步编码得到序列信息。最后，我们将编码的序列信息通过全连接层变换为输出。具体来说，我们可以将双向长短期记忆在最初时间步和最终时间步的隐藏状态连结，作为特征序列的表征传递给输出层分类。
处理步骤
1. 读入数据，并对每条记录加label
2. 分词并去掉最少词频词，创建字典
3. 设置每条记录的长度，不足的需要补齐（另一种:按长度切割每条记录并补齐最后一段）
4. 创建数据迭代器
5. 搭建模型
    Embedding实例即嵌入层，参数为字典大小、词向量维度，输出为词数，批量大小，词向量维度
    LSTM实例即为序列编码的隐藏层，输入大小为词向量维度、隐藏状态维度大小、隐藏层个数 
    Linear实例即生成分类结果的输出层。
6. 定义优化算法和损失函数
7. 训练
8. 预测


### TEXTCNN

将文本当作一维图像，从而可以用一维卷积神经网络来捕捉临近词之间的关联。

一维卷积层的工作原理：与二维卷积层一样，一维卷积层使用一维的互相关运算。在一维互相关运算中，卷积窗口从输入数组的最左方开始，按从左往右的顺序，依次在输入数组上滑动。当卷积窗口滑动到某一位置时，窗口中的输入子数组与核数组按元素相乘并求和，得到输出数组中相应位置的元素。

多输入通道的一维互相关运算也与多输入通道的二维互相关运算类似：在每个通道上，将核与相应的输入做一维互相关运算，并将通道之间的结果相加得到输出结果。

textCNN中使用的时序最大池化（max-over-time pooling）层实际上对应一维全局最大池化层：假设输入包含多个通道，各通道由不同时间步上的数值组成，各通道的输出即该通道所有时间步中最大的数值。因此，时序最大池化层的输入在各个通道上的时间步数可以不同。

为提升计算性能，我们常常将不同长度的时序样本组成一个小批量，并通过在较短序列后附加特殊字符（如0）令批量中各时序样本长度相同。这些人为添加的特殊字符当然是无意义的。由于时序最大池化的主要目的是抓取时序中最重要的特征，它通常能使模型不受人为添加字符的影响。

textCNN模型主要使用了一维卷积层和时序最大池化层。假设输入的文本序列由$n$个词组成，每个词用$d$维的词向量表示。那么输入样本的宽为$n$，高为1，输入通道数为$d$。textCNN的计算主要分为以下几步。

- 定义多个一维卷积核，并使用这些卷积核对输入分别做卷积计算。宽度不同的卷积核可能会捕捉到不同个数的相邻词的相关性。
- 对输出的所有通道分别做时序最大池化，再将这些通道的池化输出值连结为向量。
- 通过全连接层将连结后的向量变换为有关各类别的输出。这一步可以使用丢弃层应对过拟合。

